<a href="https://colab.research.google.com/github/ganeshagrahari/Pytorch-Framework/blob/main/13_RNN_using_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')

In [3]:
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [9]:
# tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()


In [11]:
tokenize("What is the capital of France?")

['what', 'is', 'the', 'capital', 'of', 'france']

In [12]:
#vocabulary
vocab = {'<UNK>' : 0}


In [19]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])
  merged_tokens  = tokenized_question + tokenized_answer

  for token in merged_tokens:
    if token not in vocab:
      vocab[token] = len(vocab)



In [20]:
df.apply(build_vocab,axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [22]:
len(vocab)

324

In [23]:
#covert word to numerical indices
def text_to_indices(text,vocab):
  indexed_text = []
  for token in tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
        indexed_text.append(vocab['<UNK>'])
  return indexed_text

In [24]:
text_to_indices("What is ganesh?",vocab)

[1, 2, 0]

In [25]:
import torch
from torch.utils.data import Dataset,DataLoader

In [26]:
class QADataset(Dataset):
  def __init__(self,df,vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self,index):
    numerical_question = text_to_indices(self.df.iloc[index]['question'],self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'],self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [27]:
dataset = QADataset(df,vocab)

In [29]:
dataset[4]

(tensor([ 1,  2,  3, 24, 25,  5, 26, 19, 27]), tensor([28]))

In [30]:
dataloader = DataLoader(dataset, batch_size = 1, shuffle = True)

In [32]:
#for question, answer in dataloader:
  #print(question,answer)

In [33]:
import torch.nn as nn

In [90]:
class SimpleRNN(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size,embedding_dim=50)
    self.rnn = nn.RNN(50,64,batch_first=True)
    self.fc = nn.Linear(64,vocab_size)


  def forward(self,question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [95]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [91]:
learning_rate = 0.001
epochs = 20

In [92]:
model = SimpleRNN(len(vocab))

In [93]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [94]:
#training loop:
for epoch in range(epochs):
  total_loss = 0
  for question, answer in dataloader:
    optimizer.zero_grad()
    #forward pass
    output = model(question)

    #output -> (1,324)-(1)
    #loss -> output->(1,324)-(1)
    loss = criterion(output,answer[0])

    #gradients
    loss.backward()

    #update
    optimizer.step()

    #loss update
    total_loss +=loss.item()

  print(f"Epoch: {epoch+1},loss : {total_loss:4f}")

Epoch: 1,loss : 522.787047
Epoch: 2,loss : 460.897602
Epoch: 3,loss : 385.505208
Epoch: 4,loss : 318.727904
Epoch: 5,loss : 265.433942
Epoch: 6,loss : 217.835855
Epoch: 7,loss : 173.895892
Epoch: 8,loss : 136.017909
Epoch: 9,loss : 104.915647
Epoch: 10,loss : 80.221209
Epoch: 11,loss : 61.574751
Epoch: 12,loss : 47.980367
Epoch: 13,loss : 38.338633
Epoch: 14,loss : 30.879969
Epoch: 15,loss : 25.277146
Epoch: 16,loss : 21.665454
Epoch: 17,loss : 18.464245
Epoch: 18,loss : 15.972644
Epoch: 19,loss : 13.837624
Epoch: 20,loss : 12.048983


In [96]:
def predict(model,question,threshold=0.5):
  #covert question to number
  numerical_question = text_to_indices(question,vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  #send to model
  output = model(question_tensor)

  # covert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  #find index of max prob
  value, index = torch.max(probs, dim=1)

  if value<threshold:
    print("I don't know")

  print(list(vocab.keys())[index])




In [97]:
predict(model, "What is the capital of France?")

paris
